In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import *

In [3]:
# 1. Create Spark session
spark = SparkSession.builder \
    .appName("KafkaToDelta") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

In [3]:
payload_schema = StructType([
    StructField("LEDGER_ENTRY_ID", StringType(), False),
    StructField("TRANSACTION_ID", StringType(), False),
    StructField("ENTRY_SEQUENCE_NO", IntegerType(), False),
    StructField("TRANSACTION_TIMESTAMP", StringType(), False),
    StructField("PROCESSING_TIMESTAMP", StringType(), False),
    StructField("VALUE_DATE", StringType(), False),
    StructField("ACCOUNT_ID", LongType(), False),
    StructField("GL_ACCOUNT_CODE", StringType(), False),
    StructField("AMOUNT", DoubleType(), False),
    StructField("CURRENCY_CODE", StringType(), False),
    StructField("ENTRY_TYPE", StringType(), False),
    StructField("EQUIVALENT_BASE_AMOUNT", DoubleType(), False),
    StructField("FX_RATE", DoubleType(), False),
    StructField("TRANSACTION_TYPE_CODE", StringType(), False),
    StructField("CHANNEL_CODE", StringType(), False),
    StructField("PROCESSING_SYSTEM_CODE", StringType(), False),
    StructField("TRANSACTION_STATUS_CODE", StringType(), False),
    StructField("ENTRY_DESCRIPTION", StringType(), True),
    StructField("BATCH_ID", StringType(), True),
    StructField("CORRELATION_ID", StringType(), True),
    StructField("IS_REVERSAL_ENTRY", BooleanType(), False),
    StructField("REVERSED_LEDGER_ENTRY_ID", StringType(), True),
    StructField("RELATED_ENTITY_TYPE", StringType(), True),
    StructField("RELATED_ENTITY_ID", StringType(), True),
    StructField("AUDIT_USER_ID", LongType(), True),
    StructField("AUDIT_CLIENT_IP", StringType(), True),
    StructField("AUDIT_HASH", StringType(), True),
    StructField("CREATED_DATE", StringType(), False),
    StructField("LAST_UPDATED_DATE", StringType(), False),
    StructField("__deleted", StringType(), True)
])


In [ ]:
# 3. Define full envelope schema for Debezium message
envelope_schema = StructType([
    StructField("schema", StructType(), True),  
    StructField("payload", payload_schema)      # actual data
])

In [ ]:
# 4. Read from Kafka topic as streaming source
raw_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "broker:29092") \
    .option("subscribe", "postgres_cdc_.core_banking.financial_ledger") \
    .option("startingOffsets", "earliest") \
    .load()

In [ ]:
# 5. Extract and parse the JSON string from Kafka 'value'
json_df = raw_df.selectExpr("CAST(value AS STRING) as json_str")

In [ ]:
# 6. Parse the JSON and extract only the payload part
payload_df = json_df \
    .withColumn("data", from_json(col("json_str"), envelope_schema)) \
    .select("data.payload.*")

In [ ]:
# 7. Write parsed payload to Delta (Bronze layer)
query = payload_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "/delta/checkpoints/financial_ledger") \
    .start("/delta/bronze/financial_ledger")

In [5]:
from delta.tables import DeltaTable
# Read the bronze Delta table
bronze_df = spark.read.format("delta").load("/delta/bronze/financial_ledger")

In [6]:
bronze_df.count()

507684

In [5]:
accounts_df.show(5)

+----------+--------------+-----------+------------+-------------+---------------+------------+------------+--------------+---------------+--------------------+--------------------+--------------------+
|account_id|account_number|customer_id|account_type|currency_code|current_balance|opening_date|closing_date|account_status|gl_account_code|        created_date|   last_updated_date|         ingested_at|
+----------+--------------+-----------+------------+-------------+---------------+------------+------------+--------------+---------------+--------------------+--------------------+--------------------+
| 100000000|  ACCT00000000|   10000000|     Savings|          USD|          37879|  2019-07-04|        null|        Closed|         101001|2025-08-02 01:35:...|2025-08-02 01:35:...|2025-08-05 08:57:...|
| 100000001|  ACCT00000001|   10000001|    Checking|          BDT|          76121|  2023-02-19|        null|        Active|         101002|2025-08-02 01:35:...|2025-08-02 01:35:...|2025-08